# MCCoder MCEval Dataset for evaluation


## Preparation


### Generate JSON dataset file from excel, python source code and log file.


#### Generate Json file from python source file


In [7]:
import pandas as pd
import os
import json

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            canonical_code = file.read()
    else:
        canonical_code = ''
    
    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = canonical_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"JSON file has been generated at {json_file_path}")


JSON file has been generated at docs/WMX3API_MCEval_Dataset.json


#### Generate Json file from python source file with motion log


In [ ]:
import pandas as pd
import os
import json
import socket
from tqdm import tqdm

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Function to modify the Python code with logging
def add_log_code(py_code, task_id, axis):
    lines = py_code.split('\n')
    modified_code = []
    i = 0
    # Add logging start code
    while i < len(lines):
        modified_code.append(lines[i])
        if '.StartHome' in lines[i]:
            i += 1
            while i < len(lines) and lines[i].strip() != '':
                modified_code.append(lines[i])
                i += 1
            if i < len(lines):
                modified_code.append(f'''
                                     
    # <log ---------------------------------------------------------------------------                                                                 
    WMX3Log = Log(Wmx3Lib)

    axislist = {axis}                           
    num = len(axislist)

    # Set Axis numbers and control variables of log
    cmLogIn_0 = CoreMotionLogInput()
    cmLogIn_0.axisSelection.axisCount = num
    for i in range(0, num):
        cmLogIn_0.axisSelection.SetAxis(i, axislist[i])

    # Control variables to log
    cmLogIn_0.axisOptions.commandPos = 1
    cmLogIn_0.axisOptions.feedbackPos = 0
    cmLogIn_0.axisOptions.commandVelocity = 0
    cmLogIn_0.axisOptions.feedbackVelocity = 0

    # Set up log time
    option = LogChannelOptions()
    option.samplingPeriodInCycles = 1
    option.samplingTimeMilliseconds = 1000000  

    ret=WMX3Log.SetLogOption(0, option)
    if ret!=0:
        print('SetLogOption error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)
    ret = WMX3Log.SetCustomLog(0,cmLogIn_0)
    if ret!=0:
        print('SetCustomLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)

    # Set log file address
    path_0 = LogFilePath()
    WMX3Log.GetLogFilePath(0)
    path_0.dirPath = "C:\\\\"
    path_0.fileName = f"{task_id}_MCEval_Log.txt"
    ret = WMX3Log.SetLogFilePath(0, path_0)
    if ret!=0:
        print('SetLogFilePath error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    # Start log
    ret = WMX3Log.StartLog(0)
    if ret!=0:
        print('StartLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.01)
    # log> ---------------------------------------------------------------------------   
    
                ''')
        i += 1

    # Add logging stop code
    i = len(lines) - 1
    while i >= 0:
        if '.SetServoOn' in lines[i]:
            while i >= 0 and lines[i].strip() != '':
                i -= 1
            if i >= 0:
                modified_code.insert(i + 1, '''
                                     
    # <log --------------------------------------------------------------------------- 
    sleep(0.1)                                                                    
    # Stop log
    ret = WMX3Log.StopLog(0)
    if ret!=0:
        print('StopLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    for axisNo in axislist:                                
        ret = Wmx3Lib_cm.home.StartHome(axisNo)
        if ret!=0:
            print('StartHome error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axisNo)                                                                  
    # log> ---------------------------------------------------------------------------     
                                     
                ''')
            break
        i -= 1

    return '\n'.join(modified_code)

# Function to send code and log data to socket server
def SendCodeLog(code_data_log):
    # 1. Create socket
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2. Connect server
    server_addr = ("10.211.55.5", 8888)
    tcp_socket.connect(server_addr)
    print("Socket connected")

    import time

    # 3. Send data
    tcp_socket.send(code_data_log.encode("utf-8"))

    time.sleep(0.1)

    send_data = 'exit'
    tcp_socket.send(send_data.encode("utf-8"))

    # 4. Receive data from server
    tcp_remsg = tcp_socket.recv(1024)
    print(tcp_remsg.decode("utf-8"))

    # 5. Close socket
    tcp_socket.close()
    print("Socket end")

# Iterate through each row in the dataframe with a progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            py_code = file.read()
    else:
        py_code = ''
    
    # Extract axis from the first row of 'EndPoint'
    try:
        axis = list(range(len(eval(row['EndPoint'])[0])))
        if not axis:
            continue
    except Exception as e:
        print(f"Error processing axis for task {task_id}: {e}")
        continue

    # Add log code to the Python file content
    modified_code = add_log_code(py_code, task_id, axis)
    
    # Save the modified code to a new file
    modified_file_path = f'MCEval_Files/{task_id}_MCEval_WithLog.py'
    with open(modified_file_path, 'w') as file:
        file.write(modified_code)
    
    # Send the modified code to the socket server for each task
    SendCodeLog(modified_code)
    
    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = py_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"JSON file has been generated at {json_file_path}")
